<a href="https://colab.research.google.com/github/roghayefazli/deep-learning-chatbot/blob/main/chatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print("\n📥      GitHub...")
!git clone https://github.com/tensorlayer/seq2seq-chatbot.git
%cd seq2seq-chatbot


📥      GitHub...
Cloning into 'seq2seq-chatbot'...
remote: Enumerating objects: 238, done.
remote: Total 238 (delta 0), reused 0 (delta 0), pack-reused 238 (from 1)
Receiving objects: 100% (238/238), 15.00 MiB | 10.57 MiB/s, done.
Resolving deltas: 100% (112/112), done.
/content/data/twitter/seq2seq-chatbot


In [ ]:
# Seq2Seq Chatbot -


# ============================================
# بخش 3: دانلود و پردازش داده‌ها
# ============================================

import os
import pickle
import re
import numpy as np
import tensorflow as tf
import tensorlayer as tl
from sklearn.utils import shuffle
from tqdm import tqdm

# دانلود دیتاست Cornell Movie
!mkdir -p data/cornell
!wget -q -O cornell_movie_dialogs.zip http://www.cs.cornell.edu/~cristian/data/cornell_movie_dialogs_corpus.zip
!unzip -q -o cornell_movie_dialogs.zip -d data/cornell/
!mv data/cornell/cornell\ movie-dialogs\ corpus/* data/cornell/
!rm cornell_movie_dialogs.zip

print("✅ data downloaded  ")

# ============================================
# بخش 4: پردازش داده‌ها
# ============================================
print("\n🔄 data process   ...")

def load_cornell_data():
    """بارگذاری و پردازش دیتاست Cornell Movie"""
    # خواندن فایل‌ها
    lines_file = 'data/cornell/movie_lines.txt'
    conv_file = 'data/cornell/movie_conversations.txt'

    # خواندن تمام خطوط
    id2line = {}
    with open(lines_file, 'r', encoding='iso-8859-1') as f:
        for line in f:
            parts = line.split(' +++$+++ ')
            if len(parts) == 5:
                id2line[parts[0]] = parts[4].strip()

    # خواندن مکالمات
    convs = []
    with open(conv_file, 'r', encoding='iso-8859-1') as f:
        for line in f:
            parts = line.split(' +++$+++ ')
            if len(parts) == 4:
                conv = eval(parts[3])
                convs.append(conv)

    # ساخت جفت‌های سوال-جواب
    questions = []
    answers = []

    for conv in convs:
        for i in range(len(conv) - 1):
            q = id2line.get(conv[i], '')
            a = id2line.get(conv[i+1], '')
            if q and a:
                questions.append(q)
                answers.append(a)

    return questions, answers

def clean_text(text):
    """تمیز کردن متن"""
    text = text.lower()
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "what is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"[-()\"#/@;:<>{}+=~|.?,]", "", text)
    return text

# بارگذاری داده‌ها
questions, answers = load_cornell_data()
print(f"✅ words : {len(questions)}")

# تمیز کردن
questions = [clean_text(q) for q in questions]
answers = [clean_text(a) for a in answers]

# ============================================
# بخش 5: ساخت واژگان
# ============================================
print("\n📚  make words  ...")

from collections import Counter

# شمارش کلمات
word_counts = Counter()
for question in questions:
    word_counts.update(question.split())
for answer in answers:
    word_counts.update(answer.split())

# انتخاب top کلمات
vocab_size = 8000
most_common = word_counts.most_common(vocab_size - 4)  # -4 برای توکن‌های خاص

# ساخت واژگان
w2idx = {'<pad>': 0, '<unk>': 1, '<start>': 2, '<end>': 3}
idx2w = {0: '<pad>', 1: '<unk>', 2: '<start>', 3: '<end>'}

for i, (word, _) in enumerate(most_common, start=4):
    w2idx[word] = i
    idx2w[i] = word

vocab_size = len(w2idx)
print(f"✅ words size : {vocab_size}")

# ============================================
# بخش 6: تبدیل متن به اعداد
# ============================================
print("\n🔢 text to number  ...")

def text_to_ids(texts, w2idx, max_len=20):
    """تبدیل متن به لیست اعداد"""
    sequences = []
    for text in texts:
        words = text.split()[:max_len]
        ids = [w2idx.get(w, w2idx['<unk>']) for w in words]
        sequences.append(ids)
    return sequences

trainX = text_to_ids(questions, w2idx)
trainY = text_to_ids(answers, w2idx)

# تقسیم به train/test
split_idx = int(len(trainX) * 0.9)
testX = trainX[split_idx:]
testY = trainY[split_idx:]
trainX = trainX[:split_idx]
trainY = trainY[:split_idx]

# تبدیل به numpy array
trainX = np.array(trainX, dtype=object)
trainY = np.array(trainY, dtype=object)
testX = np.array(testX, dtype=object)
testY = np.array(testY, dtype=object)

print(f"✅ train samples  : {len(trainX)}")
print(f"✅ test samples  : {len(testX)}")

# ذخیره داده‌ها برای استفاده بعدی
os.makedirs('data/processed', exist_ok=True)
metadata = {'w2idx': w2idx, 'idx2w': idx2w}
with open('data/processed/metadata.pkl', 'wb') as f:
    pickle.dump(metadata, f)
np.save('data/processed/trainX.npy', trainX)
np.save('data/processed/trainY.npy', trainY)
np.save('data/processed/testX.npy', testX)
np.save('data/processed/testY.npy', testY)

print("✅   data saved ")

# ============================================
# بخش 7: تنظیمات مدل
# ============================================
embedding_size = 512
hidden_size = 512
num_layers = 2
batch_size = 64
learning_rate = 0.001
num_epochs = 20  # برای شروع - می‌توانید افزایش دهید

model_dir = 'model'
os.makedirs(model_dir, exist_ok=True)

print(f"\n🎯 پارامترها: embedding={embedding_size}, hidden={hidden_size}")

# ============================================
# بخش 8: تعریف مدل Seq2Seq
# ============================================
from tensorlayer.layers import *
from tensorlayer.models import Model

class Seq2Seq(Model):
    def __init__(self):
        super(Seq2Seq, self).__init__()

        # Encoder
        self.encoder_embedding = Embedding(
            vocabulary_size=vocab_size,
            embedding_size=embedding_size,
            name='encoder_embedding'
        )

        self.encoder_rnn = RNN(
            cell=tf.keras.layers.GRUCell(hidden_size, dropout=0.1),
            return_sequences=True,
            return_state=True,
            name='encoder_rnn'
        )

        # Decoder
        self.decoder_embedding = Embedding(
            vocabulary_size=vocab_size,
            embedding_size=embedding_size,
            name='decoder_embedding'
        )

        self.decoder_rnn = RNN(
            cell=tf.keras.layers.GRUCell(hidden_size, dropout=0.1),
            return_sequences=True,
            return_state=True,
            name='decoder_rnn'
        )

        self.decoder_dense = Dense(
            n_units=vocab_size,
            name='decoder_dense'
        )

    def forward(self, encoder_inputs, decoder_inputs, initial_state=None):
        # Encoder
        encoder_embeddings = self.encoder_embedding(encoder_inputs)
        encoder_outputs, encoder_state = self.encoder_rnn(encoder_embeddings, initial_state)

        # Decoder
        decoder_embeddings = self.decoder_embedding(decoder_inputs)
        decoder_outputs, decoder_state = self.decoder_rnn(decoder_embeddings, encoder_state)
        decoder_logits = self.decoder_dense(decoder_outputs)

        return decoder_logits, decoder_state

print("\n🏗️  Seq2Seq made ")

# ============================================
# بخش 9: تابع Loss
# ============================================
def cross_entropy_seq_with_mask(logits, target_seqs, input_mask):
    """محاسبه cross-entropy با mask برای padding"""
    losses = tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits=logits,
        labels=target_seqs
    )
    losses = losses * input_mask
    loss = tf.reduce_sum(losses) / (tf.reduce_sum(input_mask) + 1e-8)
    return loss

# ============================================
# بخش 10: آموزش مدل
# ============================================
def train_model(model, trainX, trainY, num_epochs=20):
    """تابع آموزش مدل"""
    optimizer = tf.optimizers.Adam(learning_rate=learning_rate)

    n_step = len(trainX) // batch_size

    for epoch in range(num_epochs):
        trainX_shuffled, trainY_shuffled = shuffle(trainX, trainY)
        total_loss = 0

        pbar = tqdm(
            tl.iterate.minibatches(trainX_shuffled, trainY_shuffled, batch_size, shuffle=False),
            total=n_step,
            desc=f'Epoch [{epoch+1}/{num_epochs}]',
            leave=True
        )

        step = 0
        for X, Y in pbar:
            # آماده‌سازی داده‌ها
            X = tl.prepro.pad_sequences(X)
            _target_seqs = tl.prepro.sequences_add_end_id(Y, end_id=w2idx['<end>'])
            _target_seqs = tl.prepro.pad_sequences(_target_seqs)
            _decode_seqs = tl.prepro.sequences_add_start_id(Y, start_id=w2idx['<start>'], remove_last=False)
            _decode_seqs = tl.prepro.pad_sequences(_decode_seqs)
            _target_mask = tl.prepro.sequences_get_mask(_target_seqs)

            with tf.GradientTape() as tape:
                output, _ = model(X, _decode_seqs)
                loss = cross_entropy_seq_with_mask(output, _target_seqs, _target_mask)

            gradients = tape.gradient(loss, model.trainable_weights)
            optimizer.apply_gradients(zip(gradients, model.trainable_weights))

            total_loss += loss
            step += 1
            pbar.set_postfix(loss=f'{loss:.4f}')

        avg_loss = total_loss / step
        print(f"Epoch [{epoch+1}/{num_epochs}] - Average Loss: {avg_loss:.4f}")

        # ذخیره مدل
        if (epoch + 1) % 5 == 0:
            model_path = os.path.join(model_dir, f'model_epoch_{epoch+1}.npz')
            tl.files.save_npz(model.trainable_weights, model_path)
            print(f"💾 مدل ذخیره شد: {model_path}")

print("\n🚀 start training ...")
model = Seq2Seq()

# ساخت مدل با داده‌های نمونه
sample_X = np.zeros((1, 10), dtype=np.int32)
sample_Y = np.zeros((1, 10), dtype=np.int32)
_ = model(sample_X, sample_Y)

print("✅   ready !")
print(f"📊  parameters : {len(model.trainable_weights)}")

# ============================================
# بخش 11: اجرای آموزش
# ============================================
train_model(model, trainX, trainY, num_epochs=num_epochs)

# ============================================
# بخش 12: تابع تولید پاسخ (Inference)
# ============================================
def inference(seed, model, top_n=3):
    """تولید پاسخ برای یک جمله ورودی"""
    seed = clean_text(seed)
    seed_id = [w2idx.get(w, w2idx['<unk>']) for w in seed.split()]

    if not seed_id:
        return "sorry i dont understand"

    # Encode
    encoder_input = tl.prepro.pad_sequences([seed_id])
    encoder_embeddings = model.encoder_embedding(encoder_input)
    _, state = model.encoder_rnn(encoder_embeddings)

    # Decode
    decoder_input = np.array([[w2idx['<start>']]])
    sentence = []

    for _ in range(20):  # حداکثر طول پاسخ
        decoder_embeddings = model.decoder_embedding(decoder_input)
        decoder_outputs, state = model.decoder_rnn(decoder_embeddings, state)
        logits = model.decoder_dense(decoder_outputs)

        # انتخاب top_n کلمه
        probs = tf.nn.softmax(logits[0, -1]).numpy()
        top_indices = np.argsort(probs)[-top_n:]

        # انتخاب تصادفی از top_n
        word_id = np.random.choice(top_indices)

        if word_id == w2idx['<end>'] or word_id == w2idx['<pad>']:
            break

        word = idx2w[word_id]
        sentence.append(word)
        decoder_input = np.array([[word_id]])

    return ' '.join(sentence) if sentence else "..."

# ============================================
# بخش 13: تست تعاملی
# ============================================
print("\n" + "="*50)
print("🤖 chatbot is ready !")
print("="*50)
print(" questions :\n")

# تست چند نمونه
test_queries = [
    "hello",
    "how are you",
    "what is your name",
    "thank you",
    "good morning",
    "bye"
]

for query in test_queries:
    response = inference(query, model, top_n=3)
    print(f"You: {query}")
    print(f"Bot: {response}\n")

# حلقه تعاملی
print("\n💬 ask question  (  'quit' ):\n")
while True:
    try:
        query = input("You: ")
        if query.lower() in ['quit', 'exit', 'q', '']:
            print("خداحافظ! 👋")
            break

        if query.strip():
            response = inference(query, model, top_n=3)
            print(f"Bot: {response}\n")
    except:
        break

print("\n✅ اجرا تمام شد!")

✅ data downloaded  

🔄 data process   ...
✅ words : 221282

📚  make words  ...
✅ words size : 8000

🔢 text to number  ...
✅ train samples  : 199153
✅ test samples  : 22129
✅   data saved 

🎯 پارامترها: embedding=512, hidden=512

🏗️  Seq2Seq made 

🚀 start training ...


TypeError: RNN.__init__() got an unexpected keyword argument 'return_sequences'

In [ ]:
# ============================================
# بخش 2: کلون کردن ریپازیتوری
# ============================================
print("\n📥 در حال دانلود کد از GitHub...")
!rm -rf seq2seq-chatbot
!git clone https://github.com/tensorlayer/seq2seq-chatbot.git
%cd seq2seq-chatbot

# ============================================
# بخش 3: دانلود و پردازش داده‌ها
# ============================================
print("\n📥 در حال دانلود داده‌ها...")

import os
import pickle
import re
import numpy as np
import tensorflow as tf
import tensorlayer as tl
from sklearn.utils import shuffle
from tqdm import tqdm
from tensorlayer.layers import *
import tensorflow as tf
import tensorlayer as tl
from tensorlayer.layers import Embedding, Dense

# دانلود دیتاست Cornell Movie
!mkdir -p data/cornell
!wget -q -O cornell_movie_dialogs.zip http://www.cs.cornell.edu/~cristian/data/cornell_movie_dialogs_corpus.zip
!unzip -q -o cornell_movie_dialogs.zip -d data/cornell/
!mv data/cornell/cornell\ movie-dialogs\ corpus/* data/cornell/
!rm cornell_movie_dialogs.zip

print("✅ داده‌ها دانلود شدند")

# ============================================
# بخش 4: پردازش داده‌ها
# ============================================
print("\n🔄 در حال پردازش داده‌ها...")

def load_cornell_data():
    """بارگذاری و پردازش دیتاست Cornell Movie"""
    # خواندن فایل‌ها
    lines_file = 'data/cornell/movie_lines.txt'
    conv_file = 'data/cornell/movie_conversations.txt'

    # خواندن تمام خطوط
    id2line = {}
    with open(lines_file, 'r', encoding='iso-8859-1') as f:
        for line in f:
            parts = line.split(' +++$+++ ')
            if len(parts) == 5:
                id2line[parts[0]] = parts[4].strip()

    # خواندن مکالمات
    convs = []
    with open(conv_file, 'r', encoding='iso-8859-1') as f:
        for line in f:
            parts = line.split(' +++$+++ ')
            if len(parts) == 4:
                conv = eval(parts[3])
                convs.append(conv)

    # ساخت جفت‌های سوال-جواب
    questions = []
    answers = []

    for conv in convs:
        for i in range(len(conv) - 1):
            q = id2line.get(conv[i], '')
            a = id2line.get(conv[i+1], '')
            if q and a:
                questions.append(q)
                answers.append(a)

    return questions, answers

def clean_text(text):
    """تمیز کردن متن"""
    text = text.lower()
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "what is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"[-()\"#/@;:<>{}+=~|.?,]", "", text)
    return text

# بارگذاری داده‌ها
questions, answers = load_cornell_data()
print(f"✅ تعداد مکالمات: {len(questions)}")

# تمیز کردن
questions = [clean_text(q) for q in questions]
answers = [clean_text(a) for a in answers]

# ============================================
# بخش 5: ساخت واژگان
# ============================================
print("\n📚 در حال ساخت واژگان...")

from collections import Counter

# شمارش کلمات
word_counts = Counter()
for question in questions:
    word_counts.update(question.split())
for answer in answers:
    word_counts.update(answer.split())

# انتخاب top کلمات
vocab_size = 8000
most_common = word_counts.most_common(vocab_size - 4)  # -4 برای توکن‌های خاص

# ساخت واژگان
w2idx = {'<pad>': 0, '<unk>': 1, '<start>': 2, '<end>': 3}
idx2w = {0: '<pad>', 1: '<unk>', 2: '<start>', 3: '<end>'}

for i, (word, _) in enumerate(most_common, start=4):
    w2idx[word] = i
    idx2w[i] = word

vocab_size = len(w2idx)
print(f"✅ اندازه واژگان: {vocab_size}")

# ============================================
# بخش 6: تبدیل متن به اعداد
# ============================================
print("\n🔢 در حال تبدیل متن به اعداد...")

def text_to_ids(texts, w2idx, max_len=20):
    """تبدیل متن به لیست اعداد"""
    sequences = []
    for text in texts:
        words = text.split()[:max_len]
        ids = [w2idx.get(w, w2idx['<unk>']) for w in words]
        sequences.append(ids)
    return sequences

trainX = text_to_ids(questions, w2idx)
trainY = text_to_ids(answers, w2idx)

# تقسیم به train/test
split_idx = int(len(trainX) * 0.9)
testX = trainX[split_idx:]
testY = trainY[split_idx:]
trainX = trainX[:split_idx]
trainY = trainY[:split_idx]

# تبدیل به numpy array
trainX = np.array(trainX, dtype=object)
trainY = np.array(trainY, dtype=object)
testX = np.array(testX, dtype=object)
testY = np.array(testY, dtype=object)

print(f"✅ تعداد نمونه‌های آموزش: {len(trainX)}")
print(f"✅ تعداد نمونه‌های تست: {len(testX)}")

# ذخیره داده‌ها برای استفاده بعدی
os.makedirs('data/processed', exist_ok=True)
metadata = {'w2idx': w2idx, 'idx2w': idx2w}
with open('data/processed/metadata.pkl', 'wb') as f:
    pickle.dump(metadata, f)
np.save('data/processed/trainX.npy', trainX)
np.save('data/processed/trainY.npy', trainY)
np.save('data/processed/testX.npy', testX)
np.save('data/processed/testY.npy', testY)

print("✅ داده‌ها ذخیره شدند")

# ============================================
# بخش 7: تنظیمات مدل
# ============================================
embedding_size = 512
hidden_size = 512
num_layers = 2
batch_size = 64
learning_rate = 0.001
num_epochs = 20  # برای شروع - می‌توانید افزایش دهید

model_dir = 'model'
os.makedirs(model_dir, exist_ok=True)

print(f"\n🎯 پارامترها: embedding={embedding_size}, hidden={hidden_size}")

# ============================================
# بخش 8: تعریف مدل Seq2Seq
# ============================================
from tensorlayer.layers import *
from tensorlayer.models import Model

class Seq2Seq(tl.models.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(Seq2Seq, self).__init__()
        self.vocab_size = vocab_size
        self.embedding_size = embedding_size
        self.hidden_size = hidden_size

        # Embeddings (TensorLayer - بدون نیاز به in_channels)
        self.encoder_embedding = tl.layers.Embedding(vocab_size, embedding_size)
        self.decoder_embedding = tl.layers.Embedding(vocab_size, embedding_size)

        # Keras LSTM layers (بهترین سازگاری با TL 2 + TF 2)
        self.encoder_lstm = tf.keras.layers.LSTM(
            hidden_size, return_sequences=True, return_state=True
        )
        self.decoder_lstm = tf.keras.layers.LSTM(
            hidden_size, return_sequences=True, return_state=True
        )

        # Keras Dense layer (بدون نیاز به in_channels)
        self.decoder_dense = tf.keras.layers.Dense(vocab_size)

    def forward(self, encoder_inputs, decoder_inputs):
        # Encoder
        enc_emb = self.encoder_embedding(encoder_inputs)
        enc_output, enc_h, enc_c = self.encoder_lstm(enc_emb)

        # Decoder
        dec_emb = self.decoder_embedding(decoder_inputs)
        dec_output, _, _ = self.decoder_lstm(dec_emb, initial_state=[enc_h, enc_c])
        logits = self.decoder_dense(dec_output)

        return logits

print("\n🏗️ مدل Seq2Seq ساخته شد")

# ============================================
# بخش 9: تابع Loss
# ============================================
def cross_entropy_seq_with_mask(logits, target_seqs, input_mask):
    """محاسبه cross-entropy با mask برای padding"""
    losses = tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits=logits,
        labels=target_seqs
    )
    losses = losses * input_mask
    loss = tf.reduce_sum(losses) / (tf.reduce_sum(input_mask) + 1e-8)
    return loss


📥 در حال دانلود کد از GitHub...
Cloning into 'seq2seq-chatbot'...
remote: Enumerating objects: 238, done.
remote: Total 238 (delta 0), reused 0 (delta 0), pack-reused 238 (from 1)
Receiving objects: 100% (238/238), 15.00 MiB | 5.53 MiB/s, done.
Resolving deltas: 100% (112/112), done.
/content/data/twitter/seq2seq-chatbot/seq2seq-chatbot/seq2seq-chatbot/seq2seq-chatbot/seq2seq-chatbot/seq2seq-chatbot/seq2seq-chatbot/seq2seq-chatbot/seq2seq-chatbot

📥 در حال دانلود داده‌ها...
✅ داده‌ها دانلود شدند

🔄 در حال پردازش داده‌ها...
✅ تعداد مکالمات: 221282

📚 در حال ساخت واژگان...
✅ اندازه واژگان: 8000

🔢 در حال تبدیل متن به اعداد...
✅ تعداد نمونه‌های آموزش: 199153
✅ تعداد نمونه‌های تست: 22129
✅ داده‌ها ذخیره شدند

🎯 پارامترها: embedding=512, hidden=512

🏗️ مدل Seq2Seq ساخته شد


In [ ]:
def train_model(model, trainX, trainY, num_epochs=20):
    # تبدیل به لیست برای جلوگیری از مشکلات dtype=object
    trainX = [list(seq) for seq in trainX]
    trainY = [list(seq) for seq in trainY]

    optimizer = tf.optimizers.Adam(learning_rate=learning_rate)
    n_step = len(trainX) // batch_size

    # فیلتر کردن batch ناقص (اختیاری ولی توصیه‌شده)
    num_examples = n_step * batch_size
    trainX, trainY = trainX[:num_examples], trainY[:num_examples]
    n_step = len(trainX) // batch_size

    for epoch in range(num_epochs):
        trainX_shuffled, trainY_shuffled = shuffle(trainX, trainY)
        total_loss = 0

        pbar = tqdm(
            tl.iterate.minibatches(trainX_shuffled, trainY_shuffled, batch_size, shuffle=False),
            total=n_step,
            desc=f'Epoch [{epoch+1}/{num_epochs}]',
            leave=True
        )

        step = 0
        for X, Y in pbar:
            if len(X) != batch_size:
                continue  # نادیده گرفتن batch ناقص

            X = tl.prepro.pad_sequences(X)
            _target_seqs = tl.prepro.sequences_add_end_id(Y, end_id=w2idx['<end>'])
            _target_seqs = tl.prepro.pad_sequences(_target_seqs)
            _decode_seqs = tl.prepro.sequences_add_start_id(Y, start_id=w2idx['<start>'], remove_last=False)
            _decode_seqs = tl.prepro.pad_sequences(_decode_seqs)
            _target_mask = tl.prepro.sequences_get_mask(_target_seqs)

            with tf.GradientTape() as tape:
                output = model(X, _decode_seqs)  # ✅ هر دو ورودی داده شده
                loss = cross_entropy_seq_with_mask(output, _target_seqs, _target_mask)

            gradients = tape.gradient(loss, model.trainable_weights)
            optimizer.apply_gradients(zip(gradients, model.trainable_weights))

            total_loss += loss
            step += 1
            pbar.set_postfix(loss=f'{loss:.4f}')

        if step > 0:
            avg_loss = total_loss / step
            print(f"Epoch [{epoch+1}/{num_epochs}] - Average Loss: {avg_loss:.4f}")
        else:
            print("No valid batches in this epoch!")
            continue

        if (epoch + 1) % 5 == 0:
            model_path = os.path.join(model_dir, f'model_epoch_{epoch+1}.npz')
            tl.files.save_npz(model.trainable_weights, model_path)
            print(f"💾 مدل ذخیره شد: {model_path}")

In [ ]:
# تست یک batch
X_test, Y_test = next(tl.iterate.minibatches(trainX, trainY, batch_size, shuffle=False))
X_test = tl.prepro.pad_sequences(X_test)
_decode_seqs_test = tl.prepro.sequences_add_start_id(Y_test, start_id=w2idx['<start>'], remove_last=False)
_decode_seqs_test = tl.prepro.pad_sequences(_decode_seqs_test)

print("Test input shapes:", X_test.shape, _decode_seqs_test.shape)
output_test = model(X_test, _decode_seqs_test)
print("Test output shape:", output_test.shape)  # باید (batch, seq_len, vocab_size) باشد

AttributeError: 'list' object has no attribute 'shape'

In [ ]:
# Seq2Seq Chatbot - Google Colab Version (با دانلود و پردازش داده)
# نسخه تطبیق داده شده برای اجرا در Google Colab

# ============================================
# بخش 1: نصب کتابخانه‌های مورد نیاز
# ============================================




# ============================================
# بخش 10: آموزش مدل
# ============================================
def train_model(model, trainX, trainY, num_epochs=20):
    # تبدیل به لیست برای جلوگیری از مشکلات dtype=object
    trainX = [list(seq) for seq in trainX]
    trainY = [list(seq) for seq in trainY]

    optimizer = tf.optimizers.Adam(learning_rate=learning_rate)
    n_step = len(trainX) // batch_size

    # فیلتر کردن batch ناقص (اختیاری ولی توصیه‌شده)
    num_examples = n_step * batch_size
    trainX, trainY = trainX[:num_examples], trainY[:num_examples]
    n_step = len(trainX) // batch_size

    for epoch in range(num_epochs):
        trainX_shuffled, trainY_shuffled = shuffle(trainX, trainY)
        total_loss = 0

        pbar = tqdm(
            tl.iterate.minibatches(trainX_shuffled, trainY_shuffled, batch_size, shuffle=False),
            total=n_step,
            desc=f'Epoch [{epoch+1}/{num_epochs}]',
            leave=True
        )

        step = 0
        for X, Y in pbar:
            if len(X) != batch_size:
                continue  # نادیده گرفتن batch ناقص

            X = tl.prepro.pad_sequences(X)
            _target_seqs = tl.prepro.sequences_add_end_id(Y, end_id=w2idx['<end>'])
            _target_seqs = tl.prepro.pad_sequences(_target_seqs)
            _decode_seqs = tl.prepro.sequences_add_start_id(Y, start_id=w2idx['<start>'], remove_last=False)
            _decode_seqs = tl.prepro.pad_sequences(_decode_seqs)
            _target_mask = tl.prepro.sequences_get_mask(_target_seqs)

            with tf.GradientTape() as tape:
                output = model(X, _decode_seqs)  # ✅ هر دو ورودی داده شده
                loss = cross_entropy_seq_with_mask(output, _target_seqs, _target_mask)

            gradients = tape.gradient(loss, model.trainable_weights)
            optimizer.apply_gradients(zip(gradients, model.trainable_weights))

            total_loss += loss
            step += 1
            pbar.set_postfix(loss=f'{loss:.4f}')

        if step > 0:
            avg_loss = total_loss / step
            print(f"Epoch [{epoch+1}/{num_epochs}] - Average Loss: {avg_loss:.4f}")
        else:
            print("No valid batches in this epoch!")
            continue

        if (epoch + 1) % 5 == 0:
            model_path = os.path.join(model_dir, f'model_epoch_{epoch+1}.npz')
            tl.files.save_npz(model.trainable_weights, model_path)
            print(f"💾 مدل ذخیره شد: {model_path}")

print("\n🚀 شروع آموزش مدل...")
print("\n🚀 شروع آموزش مدل...")
model = Seq2Seq(vocab_size=vocab_size, embedding_size=embedding_size, hidden_size=hidden_size)
# ساخت مدل با داده‌های نمونه
sample_X = np.zeros((1, 10), dtype=np.int32)
sample_Y = np.zeros((1, 10), dtype=np.int32)
_ = model(sample_X, sample_Y)

print("✅ مدل آماده است!")
print(f"📊 تعداد پارامترها: {len(model.trainable_weights)}")

# ============================================
# بخش 11: اجرای آموزش
# ============================================
train_model(model, trainX, trainY, num_epochs=num_epochs)

# ============================================
# بخش 12: تابع تولید پاسخ (Inference)
# ============================================
def inference(seed, model, top_n=3):
    """تولید پاسخ برای یک جمله ورودی"""
    seed = clean_text(seed)
    seed_id = [w2idx.get(w, w2idx['<unk>']) for w in seed.split()]

    if not seed_id:
        return "sorry i dont understand"

    # Encode
    encoder_input = tl.prepro.pad_sequences([seed_id])
    encoder_embeddings = model.encoder_embedding(encoder_input)
    _, encoder_states = model.encoder_lstm(encoder_embeddings)

    # Decode
    decoder_input = [[w2idx['<start>']]]
    sentence = []

    for _ in range(20):  # حداکثر طول پاسخ
        decoder_input_padded = tl.prepro.pad_sequences(decoder_input)
        decoder_embeddings = model.decoder_embedding(decoder_input_padded)

        # تنظیم state
        model.decoder_lstm.initial_state = encoder_states
        decoder_outputs = model.decoder_lstm(decoder_embeddings)
        logits = model.decoder_dense(decoder_outputs)

        # انتخاب کلمه
        probs = tf.nn.softmax(logits[0, -1]).numpy()
        top_indices = np.argsort(probs)[-top_n:]
        word_id = np.random.choice(top_indices)

        if word_id == w2idx['<end>'] or word_id == w2idx['<pad>']:
            break

        word = idx2w[word_id]
        sentence.append(word)
        decoder_input[0].append(word_id)

    return ' '.join(sentence) if sentence else "..."

# ============================================
# بخش 13: تست تعاملی
# ============================================
print("\n" + "="*50)
print("🤖 چت‌بات آماده است!")
print("="*50)
print("نمونه‌هایی از سوالات:\n")

# تست چند نمونه
test_queries = [
    "hello",
    "how are you",
    "what is your name",
    "thank you",
    "good morning",
    "bye"
]

for query in test_queries:
    response = inference(query, model, top_n=3)
    print(f"You: {query}")
    print(f"Bot: {response}\n")

# حلقه تعاملی
print("\n💬 حالا شما می‌توانید سوال بپرسید (برای خروج 'quit' بزنید):\n")
while True:
    try:
        query = input("You: ")
        if query.lower() in ['quit', 'exit', 'q', '']:
            print("خداحافظ! 👋")
            break

        if query.strip():
            response = inference(query, model, top_n=3)
            print(f"Bot: {response}\n")
    except:
        break

print("\n✅ اجرا تمام شد!")


📥 در حال دانلود کد از GitHub...
Cloning into 'seq2seq-chatbot'...
remote: Enumerating objects: 238, done.
remote: Total 238 (delta 0), reused 0 (delta 0), pack-reused 238 (from 1)
Receiving objects: 100% (238/238), 15.00 MiB | 12.81 MiB/s, done.
Resolving deltas: 100% (112/112), done.
/content/data/twitter/seq2seq-chatbot/seq2seq-chatbot/seq2seq-chatbot/seq2seq-chatbot/seq2seq-chatbot/seq2seq-chatbot/seq2seq-chatbot/seq2seq-chatbot

📥 در حال دانلود داده‌ها...
✅ داده‌ها دانلود شدند

🔄 در حال پردازش داده‌ها...
✅ تعداد مکالمات: 221282

📚 در حال ساخت واژگان...
✅ اندازه واژگان: 8000

🔢 در حال تبدیل متن به اعداد...
✅ تعداد نمونه‌های آموزش: 199153
✅ تعداد نمونه‌های تست: 22129
✅ داده‌ها ذخیره شدند

🎯 پارامترها: embedding=512, hidden=512

🏗️ مدل Seq2Seq ساخته شد

🚀 شروع آموزش مدل...

🚀 شروع آموزش مدل...


TypeError: Seq2Seq.forward() missing 1 required positional argument: 'decoder_inputs'

Cornell Movie

In [ ]:
# ----------------------------------
# 📥 Cornell Movie Dialogs — Preprocessing for Colab
# Saves: idx_q.npy, idx_a.npy, metadata.pkl in current directory
# ----------------------------------

import os
import urllib.request
import zipfile
import shutil
import numpy as np
import pickle
import nltk
from collections import defaultdict
import itertools
import random

# Download required NLTK data
nltk.download('punkt', quiet=True)

# Configuration
EN_WHITELIST = '0123456789abcdefghijklmnopqrstuvwxyz '
UNK = 'unk'
VOCAB_SIZE = 8000
limit = {'minq': 2, 'maxq': 25, 'mina': 2, 'maxa': 25}

# 1. Download Cornell Movie Dialogs Corpus
print("📥 Downloading Cornell Movie Dialogs Corpus...")
url = "http://www.cs.cornell.edu/~cristian/data/cornell_movie_dialogs_corpus.zip"
zip_path = "cornell_movie_dialogs_corpus.zip"
urllib.request.urlretrieve(url, zip_path)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall("cornell_raw")

# Move data to expected path
raw_data_dir = "raw_data"
if os.path.exists(raw_data_dir):
    shutil.rmtree(raw_data_dir)
shutil.move("cornell_raw/cornell movie-dialogs corpus", raw_data_dir)
shutil.rmtree("cornell_raw")
os.remove(zip_path)

print("✅ Raw data extracted to 'raw_data/'")

# 2. Helper functions (from your data.py, adapted)
def get_id2line():
    lines = open('raw_data/movie_lines.txt', encoding='utf-8', errors='ignore').read().split('\n')
    id2line = {}
    for line in lines:
        _line = line.split(' +++$+++ ')
        if len(_line) == 5:
            id2line[_line[0]] = _line[4]
    return id2line

def get_conversations():
    conv_lines = open('raw_data/movie_conversations.txt', encoding='utf-8', errors='ignore').read().split('\n')
    convs = []
    for line in conv_lines[:-1]:
        _line = line.split(' +++$+++ ')[-1][1:-1].replace("'", "").replace(" ", "")
        convs.append(_line.split(','))
    return convs

def gather_dataset(convs, id2line):
    questions, answers = [], []
    for conv in convs:
        if len(conv) % 2 != 0:
            conv = conv[:-1]
        for i in range(len(conv)):
            if i % 2 == 0:
                questions.append(id2line[conv[i]])
            else:
                answers.append(id2line[conv[i]])
    return questions, answers

def filter_line(line, whitelist):
    return ''.join(ch for ch in line.lower() if ch in whitelist)

def filter_data(qseq, aseq):
    filtered_q, filtered_a = [], []
    assert len(qseq) == len(aseq)
    for i in range(len(qseq)):
        qlen = len(qseq[i].split())
        alen = len(aseq[i].split())
        if limit['minq'] <= qlen <= limit['maxq'] and limit['mina'] <= alen <= limit['maxa']:
            filtered_q.append(qseq[i])
            filtered_a.append(aseq[i])
    return filtered_q, filtered_a

def index_(tokenized_sentences, vocab_size):
    freq_dist = nltk.FreqDist(itertools.chain(*tokenized_sentences))
    vocab = freq_dist.most_common(vocab_size)
    index2word = ['_'] + [UNK] + [x[0] for x in vocab]
    word2index = {w: i for i, w in enumerate(index2word)}
    return index2word, word2index, freq_dist

def filter_unk(qtokenized, atokenized, w2idx):
    filtered_q, filtered_a = [], []
    for qline, aline in zip(qtokenized, atokenized):
        unk_count_a = sum(1 for w in aline if w not in w2idx)
        unk_count_q = sum(1 for w in qline if w not in w2idx)
        if unk_count_a <= 2 and (unk_count_q == 0 or unk_count_q / len(qline) <= 0.2):
            filtered_q.append(qline)
            filtered_a.append(aline)
    return filtered_q, filtered_a

def zero_pad(qtokenized, atokenized, w2idx):
    data_len = len(qtokenized)
    idx_q = np.zeros([data_len, limit['maxq']], dtype=np.int32)
    idx_a = np.zeros([data_len, limit['maxa']], dtype=np.int32)
    for i in range(data_len):
        q_idx = [w2idx.get(w, w2idx[UNK]) for w in qtokenized[i]]
        a_idx = [w2idx.get(w, w2idx[UNK]) for w in atokenized[i]]
        idx_q[i] = (q_idx + [0] * limit['maxq'])[:limit['maxq']]
        idx_a[i] = (a_idx + [0] * limit['maxa'])[:limit['maxa']]
    return idx_q, idx_a

# 3. Main preprocessing pipeline
print("⚙️  Processing data...")
id2line = get_id2line()
convs = get_conversations()
questions, answers = gather_dataset(convs, id2line)

questions = [filter_line(q, EN_WHITELIST) for q in questions]
answers = [filter_line(a, EN_WHITELIST) for a in answers]

qlines, alines = filter_data(questions, answers)

qtokenized = [[w for w in q.split() if w] for q in qlines]
atokenized = [[w for w in a.split() if w] for a in alines]

idx2w, w2idx, freq_dist = index_(qtokenized + atokenized, VOCAB_SIZE)
qtokenized, atokenized = filter_unk(qtokenized, atokenized, w2idx)

idx_q, idx_a = zero_pad(qtokenized, atokenized, w2idx)

# Save files
np.save('idx_q.npy', idx_q)
np.save('idx_a.npy', idx_a)

metadata = {
    'w2idx': w2idx,
    'idx2w': idx2w,
    'limit': limit,
    'freq_dist': freq_dist
}
with open('metadata.pkl', 'wb') as f:
    pickle.dump(metadata, f)

print(f"✅ Preprocessing complete! Dataset size: {idx_q.shape[0]}")

📥 Downloading Cornell Movie Dialogs Corpus...
✅ Raw data extracted to 'raw_data/'
⚙️  Processing data...
✅ Preprocessing complete! Dataset size: 91231


new

In [ ]:
import os
os.makedirs('data/twitter', exist_ok=True)
%cd data/twitter

# دانلود داده با لینک مستقیم Dropbox
!wget -O seq2seq.twitter.tar.gz 'https://www.dropbox.com/s/tmfwptbs3q180p0/seq2seq.twitter.tar.gz?dl=1'

# استخراج
!tar -xzf seq2seq.twitter.tar.gz

# بررسی
print("Sample lines from chat.txt:")
!head -n 6 chat.txt

/content/data/twitter
--2025-11-11 12:56:25--  https://www.dropbox.com/s/tmfwptbs3q180p0/seq2seq.twitter.tar.gz?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.18, 2620:100:6030:18::a27d:5012
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/krypoulsoiqpvjsi5k7as/seq2seq.twitter.tar.gz?rlkey=tbv0l9ehhztzcewa20zwjaut4&dl=1 [following]
--2025-11-11 12:56:25--  https://www.dropbox.com/scl/fi/krypoulsoiqpvjsi5k7as/seq2seq.twitter.tar.gz?rlkey=tbv0l9ehhztzcewa20zwjaut4&dl=1
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc51263bd0e884ef6da97afe2ff1.dl.dropboxusercontent.com/cd/0/inline/C08Jpu8sSS0dLZIxg-FKsDgk4gyhUPd2-CU6gOBrbg4wuJdXsvkW3988Al3iueefYlY3XuWC_OefW-MllBiHTSgjouI9JknXld78azrel82ryxHSR5VDa33z4etKhj1Fa-Y/file?dl=1# [following]
--2025-11-11 12:56:26--  https://uc51263bd0e884

twiter

In [ ]:
# ساخت یک فایل نمونه chat.txt
sample_lines = [
    "hi", "hello",
    "how are you", "i am fine thanks",
    "what is your name", "i am a chatbot",
    "where are you from", "i live in the cloud",
    "do you like music", "yes i love music",
    "what can you do", "i can chat with you",
    "tell me a joke", "why did the robot go to school to get a little byte",
    "are you smart", "i try my best",
    "goodbye", "see you soon",
    "thank you", "you are welcome"
]

with open('chat.txt', 'w', encoding='utf-8') as f:
    for line in sample_lines:
        f.write(line + '\n')

print("✅   'chat.txt'    made !")

✅   'chat.txt'    made !


In [ ]:
# --- نصب کتابخانه‌های مورد نیاز ---
import nltk
nltk.download('punkt', quiet=True)  # برای FreqDist و tokenization

# --- تنظیمات ---
EN_WHITELIST = '0123456789abcdefghijklmnopqrstuvwxyz '  # شامل فاصله
EN_BLACKLIST = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

# در کولب، فایل را در مسیر ریشه قرار می‌دهیم (یا آپلود می‌کنیم)
FILENAME = 'chat.txt'  # فایل باید در ریشه باشد

limit = {
    'maxq': 20,
    'minq': 1,   # تغییر: از 0 به 1 برای جلوگیری از سوال خالی
    'maxa': 20,
    'mina': 3
}

UNK = 'unk'
VOCAB_SIZE = 6000

# --- وارد کردن کتابخانه‌ها ---
import random
import sys
import nltk
import itertools
from collections import defaultdict
import numpy as np
import pickle
import os

# --- توابع کمکی ---
def read_lines(filename):
    with open(filename, 'r', encoding='utf-8', errors='ignore') as f:
        lines = f.read().split('\n')
    return [line.strip() for line in lines if line.strip()]  # حذف خطوط خالی

def filter_line(line, whitelist):
    return ''.join([ch for ch in line if ch in whitelist])

def index_(tokenized_sentences, vocab_size):
    freq_dist = nltk.FreqDist(itertools.chain(*tokenized_sentences))
    vocab = freq_dist.most_common(vocab_size)
    index2word = ['_'] + [UNK] + [x[0] for x in vocab]
    word2index = {w: i for i, w in enumerate(index2word)}
    return index2word, word2index, freq_dist

def filter_data(sequences):
    if len(sequences) % 2 != 0:
        sequences = sequences[:-1]  # حذف خط آخر اگر فرد بود
    filtered_q, filtered_a = [], []
    raw_data_len = len(sequences) // 2

    for i in range(0, len(sequences), 2):
        q = sequences[i].strip()
        a = sequences[i+1].strip()
        if not q or not a:
            continue
        qlen = len(q.split())
        alen = len(a.split())
        if limit['minq'] <= qlen <= limit['maxq'] and limit['mina'] <= alen <= limit['maxa']:
            filtered_q.append(q)
            filtered_a.append(a)

    filt_data_len = len(filtered_q)
    filtered_percent = int((raw_data_len - filt_data_len) * 100 / raw_data_len) if raw_data_len > 0 else 0
    print(f"{filtered_percent}% filtered from original data")
    return filtered_q, filtered_a

def pad_seq(seq, lookup, maxlen):
    indices = []
    for word in seq:
        indices.append(lookup.get(word, lookup[UNK]))
    indices += [0] * (maxlen - len(indices))
    return indices[:maxlen]  # برای اطمینان از طول دقیق

def zero_pad(qtokenized, atokenized, w2idx):
    data_len = len(qtokenized)
    idx_q = np.zeros([data_len, limit['maxq']], dtype=np.int32)
    idx_a = np.zeros([data_len, limit['maxa']], dtype=np.int32)

    for i in range(data_len):
        q_indices = pad_seq(qtokenized[i], w2idx, limit['maxq'])
        a_indices = pad_seq(atokenized[i], w2idx, limit['maxa'])
        idx_q[i] = q_indices
        idx_a[i] = a_indices
    return idx_q, idx_a

def process_data():
    print('\n>> Reading lines from file...')
    if not os.path.exists(FILENAME):
        print(f"❌ Error: '{FILENAME}' not found!")
        print("Please upload 'chat.txt' to Colab (use the file uploader).")
        return

    lines = read_lines(FILENAME)
    print(f"✅ Read {len(lines)} lines.")

    # تبدیل به حروف کوچک
    lines = [line.lower() for line in lines]

    # نمایش نمونه
    print('\n:: Sample lines (before filtering):')
    for i in range(min(4, len(lines))):
        print(f"  {i}: {repr(lines[i])}")

    # فیلتر کردن کاراکترها
    print('\n>> Filtering characters...')
    lines = [filter_line(line, EN_WHITELIST) for line in lines]
    lines = [' '.join(line.split()) for line in lines if line.strip()]  # حذف فاصله‌های اضافه

    print('\n:: Sample lines (after filtering):')
    for i in range(min(4, len(lines))):
        print(f"  {i}: {repr(lines[i])}")

    # فیلتر نهایی بر اساس طول
    print('\n>> Filtering by length...')
    qlines, alines = filter_data(lines)
    print(f"✅ Kept {len(qlines)} Q/A pairs.")

    if len(qlines) == 0:
        print("❌ No valid Q/A pairs found! Check your chat.txt format.")
        return

    print(f"\nExample pair:\n  Q: {qlines[0]}\n  A: {alines[0]}")

    # توکنایز
    qtokenized = [q.split() for q in qlines]
    atokenized = [a.split() for a in alines]

    # ساخت واژگان
    print('\n>> Building vocabulary...')
    idx2w, w2idx, freq_dist = index_(qtokenized + atokenized, VOCAB_SIZE)
    print(f"✅ Vocabulary size: {len(idx2w)} (max: {VOCAB_SIZE + 2})")

    # پد کردن
    print('\n>> Zero-padding sequences...')
    idx_q, idx_a = zero_pad(qtokenized, atokenized, w2idx)

    # ذخیره
    print('\n>> Saving processed data...')
    np.save('idx_q.npy', idx_q)
    np.save('idx_a.npy', idx_a)

    metadata = {
        'w2idx': w2idx,
        'idx2w': idx2w,
        'limit': limit,
        'freq_dist': freq_dist
    }
    with open('metadata.pkl', 'wb') as f:
        pickle.dump(metadata, f)

    print("✅ Done! Files saved: idx_q.npy, idx_a.npy, metadata.pkl")

def load_data(PATH=''):
    metadata = np.load(PATH + 'idx_q.npy'), np.load(PATH + 'idx_a.npy')
    with open(PATH + 'metadata.pkl', 'rb') as f:
        meta = pickle.load(f)
    return meta, metadata[0], metadata[1]

# --- اجرای اصلی ---
if __name__ == '__main__':
    process_data()

main

data